In [61]:
import requests
import pandas as pd
import json
from dateutil import parser
import constants.defs as defs

In [2]:
API_KEY = defs.API_KEY
ACCOUNT_ID = defs.ACCOUNT_ID
OANDA_URL = defs.OANDA_URL

In [3]:
session = requests.Session()

In [12]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [20]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [21]:
url = f"{OANDA_URL}/instruments/EUR_USD/candles"

In [22]:
response = session.get(url, params=params, data=None, headers=None)

In [23]:
response.status_code

200

In [24]:
response.json()

{'instrument': 'EUR_USD',
 'granularity': 'H1',
 'candles': [{'complete': True,
   'volume': 2386,
   'time': '2023-04-11T03:00:00.000000000Z',
   'bid': {'o': '1.08817', 'h': '1.08872', 'l': '1.08788', 'c': '1.08807'},
   'mid': {'o': '1.08824', 'h': '1.08879', 'l': '1.08796', 'c': '1.08815'},
   'ask': {'o': '1.08832', 'h': '1.08886', 'l': '1.08804', 'c': '1.08823'}},
  {'complete': True,
   'volume': 1729,
   'time': '2023-04-11T04:00:00.000000000Z',
   'bid': {'o': '1.08807', 'h': '1.08851', 'l': '1.08801', 'c': '1.08818'},
   'mid': {'o': '1.08814', 'h': '1.08859', 'l': '1.08808', 'c': '1.08825'},
   'ask': {'o': '1.08822', 'h': '1.08867', 'l': '1.08816', 'c': '1.08832'}},
  {'complete': True,
   'volume': 2575,
   'time': '2023-04-11T05:00:00.000000000Z',
   'bid': {'o': '1.08819', 'h': '1.08938', 'l': '1.08816', 'c': '1.08935'},
   'mid': {'o': '1.08826', 'h': '1.08945', 'l': '1.08824', 'c': '1.08944'},
   'ask': {'o': '1.08832', 'h': '1.08953', 'l': '1.08831', 'c': '1.08952'}},

In [90]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments?instruments=EUR_USD,USD_CHF"

In [91]:
response = session.get(url, params=None, headers=None)

In [92]:
response.status_code

200

In [88]:
data = response.json()

In [89]:
data.keys()

{'errorMessage': "Invalid value specified for 'instruments'",
 'errorCode': 'oanda::rest::core::InvalidParameterException'}

In [33]:
instruments_list = data['instruments']

In [35]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'minimumGuaranteedStopLossDistance', 'guaranteedStopLossOrderExecutionPremium', 'guaranteedStopLossOrderLevelRestriction', 'tags', 'financing'])

In [36]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 
         'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [75]:
instruments_dict = dict()
for i in instruments_list:
    key = i['name']
    instruments_dict[key] = {k: i[k] for k in key_i} 


In [76]:
instruments_dict

{'EUR_USD': {'name': 'EUR_USD',
  'type': 'CURRENCY',
  'displayName': 'EUR/USD',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'},
 'USD_CHF': {'name': 'USD_CHF',
  'type': 'CURRENCY',
  'displayName': 'USD/CHF',
  'pipLocation': -4,
  'displayPrecision': 5,
  'tradeUnitsPrecision': 0,
  'marginRate': '0.05'}}

In [49]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [53]:
def fetch_candles(pair_name, count, granularity='H1'):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, headers=None)
    data = response.json()

    if response.status_code == 200:
        if "candles" not in data:
            data = []
        else:
            data = data['candles']

    return response.status_code, data

In [54]:
code, data = fetch_candles('EUR_USD', 20)

In [55]:
code

200

In [56]:
len(data)

20

In [58]:
data[0]

{'complete': True,
 'volume': 2527,
 'time': '2023-04-10T18:00:00.000000000Z',
 'bid': {'o': '1.08554', 'h': '1.08588', 'l': '1.08531', 'c': '1.08587'},
 'mid': {'o': '1.08560', 'h': '1.08596', 'l': '1.08538', 'c': '1.08594'},
 'ask': {'o': '1.08566', 'h': '1.08603', 'l': '1.08543', 'c': '1.08600'}}

In [71]:

def get_candles_df(data):
    if not(data):
        return pd.DataFrame.empty
    
    final_data = []
    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']

    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"] = float(candle[p][o])

        final_data.append(new_dict)

    return pd.DataFrame.from_dict(final_data)

def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data, code)
        return
    if not(data):
        print("No candles", pair_name)
    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles {candles_df.time.min()}")

In [69]:
df = get_candles_df(data=data)

In [72]:
create_data_file('EUR_USD')

EUR_USD H1 9 candles 2023-04-11 04:00:00+00:00


In [73]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [93]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr, count=4001, granularity=g)

EUR_USD H1 4000 candles 2022-08-18 18:00:00+00:00
EUR_USD H4 4000 candles 2020-09-15 21:00:00+00:00
USD_CHF H1 4000 candles 2022-08-18 18:00:00+00:00
USD_CHF H4 4000 candles 2020-09-15 21:00:00+00:00
